In [2]:
import torch
import numpy as np
import cv2
from time import time, sleep
from ultralytics import YOLO
import supervision as sv 
from supervision.draw.color import ColorPalette
from supervision import Detections, BoxAnnotator

import smtplib
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText
from email_settings import passwords, from_email, to_email

# check if email settings is available
print(from_email)
# Advanced technique to check and send email.
def check_smtp_connection():
    global server
    try:
        server = smtplib.SMTP_SSL('smtp.googlemail.com', 465)
        fromEmail = from_email
        password = passwords
        server.login(fromEmail, password)
        return True # Connection successful
    except Exception as e :
        print(f'SMTP Connection Error: {e}')
        return False # connection failed
def send_email(from_email,passwords, people_detected = 1):
    message = MIMEMultipart()
    message['from'] = from_email
    message['to'] = to_email
    message['subject'] = 'security alert'
    # add message body 
    message.attach(MIMEText(f'Alert - {people_detected} person have been detected'))
    server.sendmail(from_email, to_email, message.as_string())

    
# check the SMTP connection
if check_smtp_connection():
    print("SMTP connection is successful. ")
    send_email(from_email, passwords)
else:
    print("SMTP connection failed. Please check your credentials or network settings.")
        

class ObjectDetection:
    def __init__(self,camera_index):
        self.camera_index = camera_index
        self.device = 'cuda' if torch.cuda.is_available() else 'cpu'
        print('using device : ', self.device)
        
        self.model = self.load_model()
        self.class_names_dict = self.model.model.names
        self.box_annotator = sv.BoxAnnotator(color = sv.ColorPalette.default(), thickness=3, text_thickness=2)
        self.email_sent = False  # Initialize email_sent attribute
        self.last_num_objects = 0  # Keep track of the number of objects in the previous frame
            
    def load_model(self):    
        model = YOLO('yolov8m.pt')
        model.fuse()
        return(model)
                     
        
    def predict(self, frame):
        results = self.model(frame)
        return(results)
    
    def plot_boxes(self, results, frame):
        bounding_boxes = []
        confidences = []
        class_ids = []
        
        # extract detection for person class 
        for result in results[0]:
            class_id = result.boxes.cls.cpu().numpy().astype(int)
            if class_id == 0: 
                bounding_boxes.append(result.boxes.xyxy.cpu().numpy())
                confidences.append(result.boxes.conf.cpu().numpy())
                class_ids.append(result.boxes.cls.cpu().numpy().astype(int))
        # setup detections for visualization
        detections = sv.Detections.from_ultralytics(results[0])
        frame = self.box_annotator.annotate(scene=frame, detections=detections)
        
        return(frame, class_ids)
            
        
             
        return(frame)
            
            
    def __call__(self):
        cap = cv2.VideoCapture(self.camera_index)
        assert cap.isOpened()
        
        cap.set(cv2.CAP_PROP_FRAME_WIDTH, 600)
        cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 400)     
        
        while True:
            start_time = time()
            ret, frame = cap.read()
            assert ret
            results = self.predict(frame)
            frame, class_ids = self.plot_boxes(results, frame)
            num_objects = len(class_ids)
            
            if num_objects != self.last_num_objects:
                if not self.email_sent:
                    send_email(from_email, to_email, num_objects)
                    self.email_sent = True
                     # Add sleep after sending email
                    sleep(3) 
                self.last_num_objects = num_objects
                
            else:
                self.email_sent = False  

            end_time = time()
            
            fps = 1/np.round(end_time - start_time, 2)
            
            
            cv2.putText(frame, f'FPS: {int(fps)}', (20, 70), cv2.FONT_HERSHEY_SIMPLEX, 1.5, (0, 255, 0), 2)
            cv2.imshow('YOLOv8 Detection', frame)
            if cv2.waitKey(5) & 0xff == 27:
                break
        cap.release()
        cv2.destroyAllWindows()
        if server:
            server.quit()
        
detector = ObjectDetection(0)
detector()        

amarkingstone20@gmail.com
SMTP connection is successful. 
using device :  cuda
YOLOv8m summary (fused): 218 layers, 25886080 parameters, 0 gradients, 78.9 GFLOPs


C:\Users\ASUS\AppData\Local\Temp\ipykernel_13320\2463189794.py:55: DeprecationWarning: default is deprecated: `ColorPalette.default()` is deprecated and will be removed in `supervision-0.20.0`. Use `Color.DEFAULT` instead.
  self.box_annotator = sv.BoxAnnotator(color = sv.ColorPalette.default(), thickness=3, text_thickness=2)



0: 384x640 7 persons, 2 bicycles, 4 cars, 1 motorcycle, 1 backpack, 28.1ms
Speed: 6.5ms preprocess, 28.1ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 6 persons, 2 bicycles, 4 cars, 1 motorcycle, 1 backpack, 190.7ms
Speed: 3.7ms preprocess, 190.7ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 7 persons, 2 bicycles, 4 cars, 1 motorcycle, 1 backpack, 73.1ms
Speed: 4.3ms preprocess, 73.1ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 7 persons, 2 bicycles, 4 cars, 1 motorcycle, 1 backpack, 27.7ms
Speed: 1.5ms preprocess, 27.7ms inference, 2.1ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 7 persons, 2 bicycles, 4 cars, 1 motorcycle, 1 backpack, 26.7ms
Speed: 3.0ms preprocess, 26.7ms inference, 3.1ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 8 persons, 2 bicycles, 4 cars, 1 motorcycle, 1 backpack, 22.4ms
Speed: 1.7ms preprocess, 22.4ms inference, 5.4ms postproc